In [2]:
%load_ext autoreload
%autoreload 2
import gc
from glob import glob
import os
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import timer, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature
import shutil

In [3]:
feim = pd.read_csv('../output/valid_single_feature_tochi.csv', header=None)
# feim = pd.read_csv('../output/valid_single_feature.csv', header=None)
feim.columns = ['datetime', 'DT-M', 'feature', 'score']

# feim = feim[feim['datetime']<='20190913_22380']
print(feim['datetime'].max())

feim.set_index('feature', inplace=True)
cnt = feim.groupby('feature')['score'].count()
idx = cnt[cnt==3].index
feim = feim.loc[idx]
may = feim[feim['DT-M']=='2018-5']
may.sort_values(by='score', ascending=False, inplace=True)
feim.loc[may.index]

20190914_07162


,datetime,DT-M,score
feature,,,
603__card3-card6_C9__ProductCD-C_std_train,20190914_06595,2018-5,0.937283
603__card3-card6_C9__ProductCD-C_std_train,20190914_06595,2018-4,0.948268
603__card3-card6_C9__ProductCD-C_std_train,20190914_06595,2018-3,0.936323
512__V130-V281__diff__ProductCD-H_train,20190914_06140,2018-5,0.937180
512__V130-V281__diff__ProductCD-H_train,20190914_06140,2018-4,0.947962
512__V130-V281__diff__ProductCD-H_train,20190914_06140,2018-3,0.935723
509__C11-V44__ratio__ProductCD-C_train,20190914_06201,2018-5,0.937060
509__C11-V44__ratio__ProductCD-C_train,20190914_06201,2018-4,0.948614
509__C11-V44__ratio__ProductCD-C_train,20190914_06201,2018-3,0.935753


In [35]:
tmp = feim[feim['DT-M']=='2018-5']
list_feature = tmp[tmp['score']>0.936].index
from_dir = 'product_feature'
# from_dir = 'check_trush'
to_dir = 'valid_use'

for feature in list_feature:
    print(feature)
    move_feature([feature.replace('_train', '')], from_dir, to_dir)

508__C13-D7__ratio__ProductCD-H_train
508__C14-D14__ratio__ProductCD-S_train
508__C3-D15__ratio__ProductCD-W_train
508__C5-D6__ratio__ProductCD-W_train
508__C7-D12__ratio__ProductCD-R_train
508__C8-D12__ratio__ProductCD-W_train
508__C9-D13__ratio__ProductCD-R_train
509__C11-V44__ratio__ProductCD-C_train
509__C11-V78__ratio__ProductCD-H_train
509__C2-V29__ratio__ProductCD-H_train
509__C4-V165__ratio__ProductCD-R_train
509__C6-V44__ratio__ProductCD-C_train
511__V48__ProductCD-C_train
512__V127-V49__ratio__ProductCD-H_train
512__V130-V281__diff__ProductCD-H_train
512__V259-V78__diff__ProductCD-R_train
512__V265-V283__diff__ProductCD-S_train
512__V265-V37__ratio__ProductCD-H_train
512__V281-V78__ratio__ProductCD-H_train
512__V29-V131__diff__ProductCD-S_train
512__V320-V306__ratio__ProductCD-C_train
512__V45-V62__ratio__ProductCD-H_train
512__V45-V67__diff__ProductCD-H_train
512__V70-V130__diff__ProductCD-W_train
512__V91-V61__diff__ProductCD-H_train
513__D13-D5__diff__ProductCD-H_train
513